In [1]:
# Load the dataset and convert to pandas
from datasets import load_dataset

ds = load_dataset("d4nieldev/qpl-completer-cot-ds")
df = ds['train'].to_pandas()
df

/Users/dani/Documents/school/finetune_llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 3043/3043 [00:00<00:00, 501149.17 examples/s]


,db_id,parent_question,question,prefix_qpl,op,cot,qpl_line
0,department_management,How many heads of the departments are older th...,List 1 for each head of the departments that o...,,Scan,## Understanding the Input Stream\n\nSince the...,#1 = Scan Table [ head ] Predicate [ age > 56....
1,department_management,None,How many heads of the departments are older th...,#1 = Scan Table [ head ] Predicate [ age > 56....,Aggregate,## Understanding the Input Stream\n\nThe opera...,#2 = Aggregate [ #1 ] Output [ countstar AS Co...
2,department_management,"List the name, born state and age of the heads...","List the name, born state and age of the heads...",,Scan,## Understanding the Input Stream\n\nSince the...,"#1 = Scan Table [ head ] Output [ name , born_..."
3,department_management,None,"List the name, born state and age of the heads...","#1 = Scan Table [ head ] Output [ name , born_...",Sort,## Understanding the Input Stream\n\nThe input...,#2 = Sort [ #1 ] OrderBy [ age ASC ] Output [ ...
4,department_management,None,"List the creation year, name and budget of eac...",,Scan,## Understanding the Input Stream\n\nSince the...,#1 = Scan Table [ department ] Output [ Creati...
...,...,...,...,...,...,...,...
10804,culture_company,What are the titles of movies and books corres...,What are the ids and titles of all movies?,,Scan,## Understanding the Input Stream\n\nSince the...,"#4 = Scan Table [ movie ] Output [ movie_id , ..."
10805,culture_company,None,What are the titles of movies and books corres...,#1 = Scan Table [ culture_company ] Predicate ...,Join,## Understanding the Input Stream\n\nThe final...,"#5 = Join [ #3 , #4 ] Predicate [ #3.movie_id ..."
10806,culture_company,Show all company names with a movie directed i...,Show all movie ids directed in year 1999.,,Scan,## Understanding the Input Stream\n\nSince the...,#1 = Scan Table [ movie ] Predicate [ Year = 1...
10807,culture_company,Show all company names with a movie directed i...,Show the names and movie ids of all companies.,,Scan,## Understanding the Input Stream\n\nSince the...,#2 = Scan Table [ culture_company ] Output [ C...


In [3]:
# Convert 'Top' to 'TopSort'
df = df[df['op'] != 'Top']
print(df['op'].value_counts())

op
Scan         6016
Join         1705
Aggregate    1673
TopSort       533
Sort          288
Filter        225
Except        221
Intersect     131
Union          16
Name: count, dtype: int64


In [7]:
# Over/Under-sample the dataset
import pandas as pd

# Over/Under-sample ratios
ratios = {
    'Scan': 1,
    'Aggregate': 1,
    'Sort': 1,
    'Join': 1,
    'Filter': 2,
    'TopSort': 1,
    'Except': 2,
    'Intersect': 2,
    'Union': 5
}

def sample_group(group):
    tag = group['op'].iloc[0]
    ratio = ratios.get(tag, 1.0)
    n = len(group)
    target = int(round(n * ratio))
    if ratio < 1:
        return group.sample(frac=ratio)
    elif ratio > 1:
        full_repeats = target // n
        rest = target % n
        parts = [group] * full_repeats
        if rest > 0:
            parts.append(group.sample(n=rest))
        return pd.concat(parts)
    else:
        return group

sampled_groups = pd.concat([sample_group(g) for _, g in df.groupby('op')])
balanced_df = sampled_groups.sample(frac=1).reset_index(drop=True)
print(balanced_df['op'].value_counts())

op
Scan         6016
Join         1705
Aggregate    1673
TopSort       533
Filter        450
Except        442
Sort          288
Intersect     262
Union          80
Name: count, dtype: int64


In [8]:
merged = df.merge(balanced_df.drop_duplicates(), how='left', indicator=True)
all_present = ((merged['_merge'] == 'both') | (merged['op'] == 'Scan')).all()
print(all_present)

True


In [11]:
from datasets import Dataset

ds['train'] = Dataset.from_pandas(balanced_df)
ds.push_to_hub("d4nieldev/qpl-completer-cot-ds", "balanced")




Creating parquet from Arrow format: 100%|██████████| 12/12 [00:00<00:00, 236.99ba/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.60s/ shards]



Creating parquet from Arrow format: 100%|██████████| 4/4 [00:00<00:00, 279.78ba/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.61 shards/s]


CommitInfo(commit_url='https://huggingface.co/datasets/d4nieldev/qpl-completer-cot-ds/commit/28447f59028c8db2921e1d4671dea8156d6577d7', commit_message='Upload dataset', commit_description='', oid='28447f59028c8db2921e1d4671dea8156d6577d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/d4nieldev/qpl-completer-cot-ds', endpoint='https://huggingface.co', repo_type='dataset', repo_id='d4nieldev/qpl-completer-cot-ds'), pr_revision=None, pr_num=None)